In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/processed_tweets_binary.csv')

In [4]:
from sklearn.model_selection import train_test_split

X = df["processed_tweet"]
y = df["emotion_encoded"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2483,), (1065,), (2483,), (1065,))

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

vectorizer = TfidfVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Convert to dense matrix
X_train_vec_dense = X_train_vec.todense()
X_test_vec_dense = X_test_vec.todense()

# Model
model = Sequential()
model.add(Dense(10, input_dim=X_train_vec_dense.shape[1], activation='relu'))
model.add(Dense(2, activation='sigmoid'))  # Assuming 3 classes

# Compilation
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

# Training
model.fit(X_train_vec_dense, y_train_encoded, epochs=10, batch_size=5)

Epoch 1/10
497/497 [==============================] - 0s 851us/step - loss: 0.4662 - accuracy: 0.8401
Epoch 2/10
497/497 [==============================] - 0s 776us/step - loss: 0.3566 - accuracy: 0.8405
Epoch 3/10
497/497 [==============================] - 0s 781us/step - loss: 0.2808 - accuracy: 0.8562
Epoch 4/10
497/497 [==============================] - 0s 764us/step - loss: 0.2078 - accuracy: 0.9134
Epoch 5/10
497/497 [==============================] - 0s 772us/step - loss: 0.1506 - accuracy: 0.9573
Epoch 6/10
497/497 [==============================] - 0s 787us/step - loss: 0.1087 - accuracy: 0.9762
Epoch 7/10
497/497 [==============================] - 0s 798us/step - loss: 0.0802 - accuracy: 0.9835
Epoch 8/10
497/497 [==============================] - 0s 794us/step - loss: 0.0608 - accuracy: 0.9883
Epoch 9/10
497/497 [==============================] - 0s 802us/step - loss: 0.0469 - accuracy: 0.9899
Epoch 10/10
497/497 [==============================] - 0s 777us/step - loss: 0.036

In [6]:
from sklearn.metrics import classification_report

preds = np.argmax(model.predict(X_test_vec_dense), axis=1)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.69      0.43      0.53       173
           1       0.90      0.96      0.93       892

    accuracy                           0.88      1065
   macro avg       0.79      0.69      0.73      1065
weighted avg       0.86      0.88      0.86      1065

